In [1]:
import numpy as np
import os
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import tensorflow as tf

In [3]:
print(tf.__version__)

1.3.0


## Load data

In [4]:
DATASET_PATH = "./dataset/UCI HAR Dataset/"

In [5]:
INPUT_SIGNAL_TYPES = [
    "body_acc_x_",
    "body_acc_y_",
    "body_acc_z_",
    "body_gyro_x_",
    "body_gyro_y_",
    "body_gyro_z_",
    "total_acc_x_",
    "total_acc_y_",
    "total_acc_z_"
]

In [6]:
def load_X(X_signals_paths):
    X_signals = []
    
    for signal_type_path in X_signals_paths:
        with open(signal_type_path, "r") as f:
            X_signals.append(
                [np.array(serie, dtype=np.float32)
                    for serie in [row.replace('  ', ' ').strip().split(' ') for row in f]]
            )
    
    return np.transpose(X_signals, (1, 2, 0))

def load_y(y_path):

    # Read dataset from disk, dealing with text file's syntax
    with open(y_path, "r") as f:
        y = np.array(
            [elem for elem in [
                row.replace('  ', ' ').strip().split(' ') for row in f
            ]], 
            dtype=np.int32
        )
            
    y = y.reshape(-1,)
    # Substract 1 to each output class for friendly 0-based indexing 
    return y - 1

In [7]:
X_train_signals_paths = [
    DATASET_PATH + "train/Inertial Signals/" + signal + "train.txt" for signal in INPUT_SIGNAL_TYPES
]
X_test_signals_paths = [
    DATASET_PATH + "test/Inertial Signals/" + signal + "test.txt" for signal in INPUT_SIGNAL_TYPES
]

In [8]:
har_X_train = load_X(X_train_signals_paths)
har_X_test = load_X(X_test_signals_paths)

In [9]:
y_train_path = DATASET_PATH + "train/y_train.txt"
y_test_path = DATASET_PATH + "test/y_test.txt"

In [10]:
har_y_train = load_y(y_train_path)
har_y_test = load_y(y_test_path)

In [11]:
har_X_train[0,:5,:]

array([[  1.80851493e-04,   1.07668098e-02,   5.55606782e-02,
          3.01912203e-02,   6.60136193e-02,   2.28586402e-02,
          1.01281703e+00,  -1.23216704e-01,   1.02934100e-01],
       [  1.01385601e-02,   6.57948013e-03,   5.51248305e-02,
          4.37107086e-02,   4.26989682e-02,   1.03157200e-02,
          1.02283299e+00,  -1.26875594e-01,   1.05687201e-01],
       [  9.27557424e-03,   8.92887823e-03,   4.84047309e-02,
          3.56878005e-02,   7.48501793e-02,   1.32496897e-02,
          1.02202797e+00,  -1.24003701e-01,   1.02102503e-01],
       [  5.06589701e-03,   7.48868287e-03,   4.97749709e-02,
          4.04020995e-02,   5.73197417e-02,   1.77512094e-02,
          1.01787698e+00,  -1.24927901e-01,   1.06552698e-01],
       [  1.08102504e-02,   6.14096597e-03,   4.30131406e-02,
          4.70965393e-02,   5.23428395e-02,   2.55336706e-03,
          1.02367997e+00,  -1.25766695e-01,   1.02813497e-01]], dtype=float32)

## Prepare data

In [12]:
X_train, X_valid, y_train, y_valid = train_test_split(har_X_train, har_y_train, random_state=42)

In [13]:
X_train.shape, y_train.shape

((5514, 128, 9), (5514,))

In [14]:
X_valid.shape

(1838, 128, 9)

In [15]:
har_X_test.shape

(2947, 128, 9)

In [16]:
har_X_test[0][:2]

array([[ 0.01165315, -0.02939904,  0.1068262 ,  0.4374637 ,  0.53134918,
         0.1365279 ,  1.04121602, -0.26979589,  0.02377977],
       [ 0.01310909, -0.03972867,  0.1524549 ,  0.4682641 ,  0.7210685 ,
         0.09762239,  1.041803  , -0.28002501,  0.07629271]], dtype=float32)

In [17]:
np.amax(har_y_test)

5

In [21]:
np.amax(har_y_train)

5

In [22]:
from collections import Counter

In [23]:
c_test = Counter(har_y_test)

In [24]:
c_test

Counter({0: 496, 1: 471, 2: 420, 3: 491, 4: 532, 5: 537})

In [25]:
c_train = Counter(har_y_train)
c_train

Counter({0: 1226, 1: 1073, 2: 986, 3: 1286, 4: 1374, 5: 1407})

# Build CNN model

## Hyperparameters

In [18]:
batch_size = 200
learning_rate = 0.001
n_epochs = 100

# outputs
n_classes = 6

seq_len = 128
n_channels = 9

In [19]:
n_batches = X_train.shape[0] // batch_size

## first CNN model

In [20]:
tf.reset_default_graph()

X = tf.placeholder(tf.float32, (None, seq_len, n_channels), name="X")
y = tf.placeholder(tf.int32, (None), name="y")

# is in training phase
training = tf.placeholder_with_default(False, shape=(), name="training")


In [21]:
with tf.name_scope("cnn"):
    
    # Convolutional layers
    conv1 = tf.layers.conv1d(X, 
                             filters=32, 
                             kernel_size=2, 
                             strides=1,
                             padding="same",
                             activation=tf.nn.relu)

    conv2 = tf.layers.conv1d(conv1,
                             filters=64, 
                             kernel_size=2,
                             strides=1,
                             padding="same",
                             activation=tf.nn.relu)

    
    last_shape = conv2.get_shape().as_list()
    print(last_shape)
    # Fully Connected layers
    flat = tf.reshape(conv2, (-1, last_shape[1] * last_shape[2]))
   
    fc1 = tf.layers.dense(flat, 100, activation=tf.nn.relu)
    fc2 = tf.layers.dense(fc1, 100, activation=tf.nn.relu)
    
    logits = tf.layers.dense(fc2, n_classes)
    
    

[None, 128, 64]


In [22]:
logits.get_shape()

TensorShape([Dimension(None), Dimension(6)])

In [23]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)
    loss = tf.reduce_mean(xentropy, name="loss")

In [24]:
with tf.name_scope("train"):
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    training_op = optimizer.minimize(loss)

In [25]:
with tf.name_scope("eval"):
    #correct = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

In [26]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [27]:
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        shuffled_idx = np.random.permutation(X_train.shape[0])
        X_random = X_train[shuffled_idx]
        y_random = y_train[shuffled_idx]
        for i in range(n_batches):
            X_batch = X_random[i * batch_size:(i+1) * batch_size]
            y_batch = y_random[i * batch_size:(i+1) * batch_size]
            train_acc =  sess.run([training_op], feed_dict={X: X_batch, y: y_batch, training: True})
        
        
        if epoch % 10 == 0:
            valid_acc, loss_val = sess.run([accuracy, loss], feed_dict={X: X_valid, y: y_valid})
            print("Epoch: ", epoch, "Valid accuracy: ", valid_acc)
    saver.save(sess, "./model/har_cnn_model.ckpt")

Epoch:  0 Valid accuracy:  0.797062
Epoch:  10 Valid accuracy:  0.947225
Epoch:  20 Valid accuracy:  0.949401
Epoch:  30 Valid accuracy:  0.959195
Epoch:  40 Valid accuracy:  0.965724
Epoch:  50 Valid accuracy:  0.96518
Epoch:  60 Valid accuracy:  0.971164
Epoch:  70 Valid accuracy:  0.970076
Epoch:  80 Valid accuracy:  0.966812
Epoch:  90 Valid accuracy:  0.975517


In [29]:
with tf.Session() as sess:
    saver.restore(sess, "./model/har_cnn_model.ckpt")
    valid_acc = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
    test_acc = accuracy.eval(feed_dict={X: har_X_test, y: har_y_test})
    print("Valid Accuracy: ", valid_acc)
    print("Test Accuracy: ", test_acc)

INFO:tensorflow:Restoring parameters from ./model/har_cnn_model.ckpt
Test Accuracy:  0.891415


## CNN model with learning rate decay and batch norm

In [30]:
tf.reset_default_graph()

X = tf.placeholder(tf.float32, (None, seq_len, n_channels), name="X")
y = tf.placeholder(tf.int32, (None), name="y")

# is in training phase
training = tf.placeholder_with_default(False, shape=(), name="training")

# learning rate exponential decay
global_step = tf.Variable(0, trainable=False)
starter_learning_rate = 0.01
learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
                                           100, 0.96, staircase=True)

In [31]:
with tf.name_scope("cnn"):
    
    # Convolutional layers
    conv1 = tf.layers.conv1d(X, 
                             filters=32, 
                             kernel_size=2, 
                             strides=1,
                             padding="same",
                             activation=tf.nn.relu)
    max_pool_1 = tf.layers.max_pooling1d(inputs=conv1, pool_size=4, strides=2, padding='same')

    conv2 = tf.layers.conv1d(max_pool_1,
                             filters=64, 
                             kernel_size=2,
                             strides=1,
                             padding="same",
                             activation=tf.nn.relu)
    max_pool_2 = tf.layers.max_pooling1d(inputs=conv2, pool_size=4, strides=2, padding='same')

    conv3 = tf.layers.conv1d(max_pool_2,
                             filters=96, 
                             kernel_size=2,
                             strides=1,
                             padding="same",
                             activation=tf.nn.relu)
    max_pool_3 = tf.layers.max_pooling1d(inputs=conv3, pool_size=4, strides=2, padding='same')
    
    last_shape = max_pool_3.get_shape().as_list()
    print(last_shape)
    # Fully Connected layers
    flat = tf.reshape(max_pool_3, (-1, last_shape[1] * last_shape[2]))
    #drop = tf.nn.dropout(flat, keep_prob=1.0, training=training)
    
    fc1 = tf.layers.dense(flat, 100)
    bn_fc1 = tf.layers.batch_normalization(fc1, training=training)
    bn_fc1_act = tf.nn.relu(bn_fc1)
    
    fc2 = tf.layers.dense(bn_fc1_act, 100)
    bn_fc2 = tf.layers.batch_normalization(fc2, training=training)
    bn_fc2_act = tf.nn.relu(bn_fc2)
    
    logits_before_bn = tf.layers.dense(bn_fc2_act, n_classes)
    logits = tf.layers.batch_normalization(logits_before_bn, training=training)
    #logits = tf.layers.dense(bn_fc2_act, n_classes)
    
    

[None, 16, 96]


In [32]:
logits.get_shape()

TensorShape([Dimension(None), Dimension(6)])

In [33]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)
    loss = tf.reduce_mean(xentropy, name="loss")

In [34]:
with tf.name_scope("train"):
    extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    with tf.control_dependencies(extra_update_ops):
        optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
        training_op = optimizer.minimize(loss, global_step=global_step)

In [35]:
with tf.name_scope("eval"):
    #correct = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

In [36]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [37]:
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        shuffled_idx = np.random.permutation(X_train.shape[0])
        X_random = X_train[shuffled_idx]
        y_random = y_train[shuffled_idx]
        for i in range(n_batches):
            X_batch = X_random[i * batch_size:(i+1) * batch_size]
            y_batch = y_random[i * batch_size:(i+1) * batch_size]
            train_acc =  sess.run([training_op], feed_dict={X: X_batch, y: y_batch, training: True})
        
        
        if epoch % 10 == 0:
            valid_acc, loss_val = sess.run([accuracy, loss], feed_dict={X: X_valid, y: y_valid})
            print("Epoch: ", epoch,"Learning rate:", sess.run(optimizer._lr), "Loss:", loss_val, "Valid accuracy: ", valid_acc)
    saver.save(sess, "./model/har_cnn_model_w_batch_norm.ckpt")

Epoch:  0 Learning rate: 0.01 Valid accuracy:  0.143634
Epoch:  10 Learning rate: 0.009216 Valid accuracy:  0.863439
Epoch:  20 Learning rate: 0.00815373 Valid accuracy:  0.850381
Epoch:  30 Learning rate: 0.00721389 Valid accuracy:  0.818281
Epoch:  40 Learning rate: 0.00638239 Valid accuracy:  0.780196
Epoch:  50 Learning rate: 0.00588201 Valid accuracy:  0.958651
Epoch:  60 Learning rate: 0.00520403 Valid accuracy:  0.930903
Epoch:  70 Learning rate: 0.00460419 Valid accuracy:  0.968988
Epoch:  80 Learning rate: 0.00424322 Valid accuracy:  0.967356
Epoch:  90 Learning rate: 0.00375413 Valid accuracy:  0.972796


In [39]:
with tf.Session() as sess:
    saver.restore(sess, "./model/har_cnn_model_w_batch_norm.ckpt")
    valid_acc = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
    test_acc = accuracy.eval(feed_dict={X: har_X_test, y: har_y_test})
    print("Valid accuracy: ", valid_acc)
    print("Test Accuracy: ", test_acc)

INFO:tensorflow:Restoring parameters from ./model/har_cnn_model_w_batch_norm.ckpt
Valid accuracy:  0.941785
Test Accuracy:  0.903291


## CNN model with more regularization

In [182]:
dropout_rate=0.3
n_epochs = 800

In [183]:
tf.reset_default_graph()

X = tf.placeholder(tf.float32, (None, seq_len, n_channels), name="X")
y = tf.placeholder(tf.int32, (None), name="y")

# is in training phase
training = tf.placeholder_with_default(False, shape=(), name="training")

# learning rate exponential decay
global_step = tf.Variable(0, trainable=False)
starter_learning_rate = 0.0001
learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
                                           400, 0.96, staircase=True)

In [184]:
with tf.name_scope("cnn"):
    
    # Convolutional layers
    conv1 = tf.layers.conv1d(X, 
                             filters=96, 
                             kernel_size=2, 
                             strides=1,
                             padding="same",
                             activation=tf.nn.relu)
    max_pool_1 = tf.layers.max_pooling1d(inputs=conv1, pool_size=2, strides=2, padding='same')
    #drop1 = tf.layers.dropout(max_pool_1, dropout_rate, training=training)
    
    conv2 = tf.layers.conv1d(max_pool_1,
                             filters=256, 
                             kernel_size=2,
                             strides=1,
                             padding="same",
                             activation=tf.nn.relu)
    max_pool_2 = tf.layers.max_pooling1d(inputs=conv2, pool_size=2, strides=2, padding='same')
    #drop2 = tf.layers.dropout(max_pool_2, dropout_rate, training=training)
    
    conv3 = tf.layers.conv1d(max_pool_2,
                             filters=96, 
                             kernel_size=5,
                             strides=1,
                             padding="same",
                             activation=tf.nn.relu)
    max_pool_3 = tf.layers.max_pooling1d(inputs=conv3, pool_size=2, strides=2, padding='same')
    #drop3 = tf.layers.dropout(max_pool_3, dropout_rate, training=training)
    
    last_shape = max_pool_3.get_shape().as_list()
    print(last_shape)
    # Fully Connected layers
    flat = tf.reshape(max_pool_3, (-1, last_shape[1] * last_shape[2]))
    
    
    fc1 = tf.layers.dense(flat, 300, kernel_regularizer=tf.contrib.layers.l2_regularizer(0.0001))
    bn_fc1 = tf.layers.batch_normalization(fc1, training=training)
    bn_fc1_act = tf.nn.relu(bn_fc1)
    drop1 = tf.layers.dropout(bn_fc1_act, dropout_rate, training=training)
    
    fc2 = tf.layers.dense(drop1, 300, kernel_regularizer=tf.contrib.layers.l2_regularizer(0.0001))
    bn_fc2 = tf.layers.batch_normalization(fc2, training=training)
    bn_fc2_act = tf.nn.relu(bn_fc2)
    drop2 = tf.layers.dropout(bn_fc2_act, dropout_rate, training=training)
    
    logits_before_bn = tf.layers.dense(drop2, n_classes)
    logits = tf.layers.batch_normalization(logits_before_bn, training=training)
    #logits = tf.layers.dense(flat, n_classes)
    
    

[None, 16, 96]


In [185]:
logits.get_shape()

TensorShape([Dimension(None), Dimension(6)])

In [186]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)
    reg_loss = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
    loss = tf.add_n(reg_loss + [tf.reduce_mean(xentropy)], name="loss")

In [187]:
with tf.name_scope("train"):
    extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    with tf.control_dependencies(extra_update_ops):
        #optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9)
        optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
        training_op = optimizer.minimize(loss, global_step=global_step)

In [188]:
with tf.name_scope("eval"):
    #correct = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

In [189]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [191]:
best_loss = np.infty
epochs_without_progress = 0
max_epochs_without_progress=50

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        shuffled_idx = np.random.permutation(X_train.shape[0])
        X_random = X_train[shuffled_idx]
        y_random = y_train[shuffled_idx]
        
        for i in range(n_batches):
            X_batch = X_random[i * batch_size:(i+1) * batch_size]
            y_batch = y_random[i * batch_size:(i+1) * batch_size]
            train_acc, loss_val =  sess.run([training_op, loss], feed_dict={X: X_batch, y: y_batch, training: True})
        
        
        if epoch % 5 == 0:
            valid_acc = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
            print("Epoch: ", epoch, "Loss: ", loss_val, "Valid accuracy: ", valid_acc)
            if np.around(loss_val, decimals=3) < best_loss:
                saver.save(sess, "./model/har_cnn_model_more_reg.ckpt")
                best_loss = loss_val
            else:
                epochs_without_progress +=5
                if epochs_without_progress > max_epochs_without_progress:
                    break
                    
    print("Best Loss: ", best_loss)

Epoch:  0 Loss:  1.33341 Valid accuracy:  0.182807
Epoch:  5 Loss:  0.821939 Valid accuracy:  0.616975
Epoch:  10 Loss:  0.675122 Valid accuracy:  0.92111
Epoch:  15 Loss:  0.566769 Valid accuracy:  0.945593
Epoch:  20 Loss:  0.594539 Valid accuracy:  0.952122
Epoch:  25 Loss:  0.474179 Valid accuracy:  0.951034
Epoch:  30 Loss:  0.49471 Valid accuracy:  0.955386
Epoch:  35 Loss:  0.490799 Valid accuracy:  0.958107
Epoch:  40 Loss:  0.435741 Valid accuracy:  0.957018
Epoch:  45 Loss:  0.511263 Valid accuracy:  0.958107
Epoch:  50 Loss:  0.448011 Valid accuracy:  0.957562
Epoch:  55 Loss:  0.442497 Valid accuracy:  0.957563
Epoch:  60 Loss:  0.411824 Valid accuracy:  0.955386
Epoch:  65 Loss:  0.415154 Valid accuracy:  0.957563
Epoch:  70 Loss:  0.436305 Valid accuracy:  0.957562
Epoch:  75 Loss:  0.393108 Valid accuracy:  0.958107
Epoch:  80 Loss:  0.416108 Valid accuracy:  0.960827
Epoch:  85 Loss:  0.399803 Valid accuracy:  0.961371
Epoch:  90 Loss:  0.370778 Valid accuracy:  0.96354

In [192]:
with tf.Session() as sess:
    saver.restore(sess, "./model/har_cnn_model_more_reg.ckpt")
    valid_acc = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
    test_acc = accuracy.eval(feed_dict={X: har_X_test_normed, y: har_y_test})
    print("Valid accuracy: ", valid_acc)
    print("Test Accuracy: ", test_acc)

INFO:tensorflow:Restoring parameters from ./model/har_cnn_model.ckpt
Valid accuracy:  0.963547
Test Accuracy:  0.938242
